<a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/TareaSemana1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![MAIA banner](https://raw.githubusercontent.com/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/main/Images/Aprendizaje_refuerzo_profundo_Banner_V1.png)

# <h1><center>Tarea Tutorial - Semana 1 <a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/TareaSemana1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

<center><h1>Aproximación de Funciones</h1></center>

En este tutorial aprenderás acerca de las ventajas que tiene utilizar aproximación de funciones en problemas de aprendizaje por refuerzo profundo. Ilustraremos el proceso con el problema de <a href="https://gymnasium.farama.org/environments/classic_control/cart_pole/">Cart Pole</a>, incluido en las librerías de Gym, e implementaremos redes nuronales con el framework <a href="https://github.com/inarikami/keras-rl2/blob/master/examples/dqn_cartpole.py">keras-rl2</a>, que ya incluye varios algoritmos de aprendizaje por refuerzo. El objetivo principal será comparar el desempeño de distintos algoritmos en su versión tabular y su versión con redes neuronales. Este notebook tutorial se divide en las siguientes secciones:


# Tabla de Contenidos
1. [Objetivos de Aprendizaje](#scrollTo=Objetivos_de_Aprendizaje)  
2. [Marco Teórico](#scrollTo=Marco_Te_rico)  
3. [Instalación de Librerías](#scrollTo=Instalaci_n_de_Librer_as)  
4. [Familiarización con el Entorno de Gym](#scrollTo=Familiarizaci_n_con_el_Entorno_de_Gym)  
5. [Métodos Tabulares](#scrollTo=M_todos_Tabulares)  
6. [Métodos con Redes Neuronales](#scrollTo=M_todos_con_Redes_Neuronales)  
7. [Comparación](#scrollTo=Comparaci_n)  
8. [Conclusiones](#scrollTo=Conclusiones)  
9. [Referencias](#scrollTo=Referencias)

## Objetivos de Aprendizaje  
  
* Implementar métodos de aprendizaje por refuerzo de forma existosa con redes neuronales (Deep SARSA y Deep Q-Networks).
* Familiarizarse con los entornos de simulación de Gym.
* Identificar las ventajas de implementar aproximación funciones en comparación a utilizar una versión tabular de los algoritmos.


## Marco Teórico  
En el aprendizaje por refuerzo hay dos grandes familias de métodos para encontrar políticas de comportamiento óptimas: los métodos tabulares y los métodos por aproximación de funciones. Entre los métodos tabulares algunos de los algoritmos más utilizados son los de SARSA y Q-Learning. En la aproximación de funciones mediante redes neuronales estos dos métodos encuentran su equivalente en la forma de Deep SARSA y Deep Q-Networks (DQN).


![Interaccion_agente_ambiente](https://raw.githubusercontent.com/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/main/Images/Interaccion_agente_ambiente.png)

<center>Figura 1. Interacción de un agente con el ambiente en un problema de aprendizaje por refuerzo. [1]</center>


Recordemos que un ambiente en un problema de aprendizaje por refuerzo está definido por estados, acciones y recompensas. En casa estado $S_t$ el agente toma una acción $A_t$, y tiene cierta probabilidad de pasar a otro estado $S_{t+1}$ y recibir una recompensa $R_{t+1}$. El objetivo del aprendizaje por refuerzo es hallar una política de comportamiento tal que en cada estado el agente escoja la mejor acción posible para maximizar el retorno obtenido a largo plazo. Precisamente, esto es lo que indica la función de valor de parejas estado-acción $Q(S_t,A_t)$. La función $Q$ nos dice cuál es el retorno esperado si es un estado $S_t$ se toma una acción $A_t$ siguiendo una política específica $\pi$. SARSA y Q-Learning son métodos que iterativamente calculan esta función $Q$ de acuerdo con las siguientes reglas de actualización:

<center> $Q(S_t,A_t) \ ← \ Q(S_t,A_t)+α[R_{t+1}+\gamma Q(S_{t+1},A_{t+1})-Q(S_t,A_t)]$ &emsp;&emsp;&emsp;$(1)$ </center>

<center> $Q(S_t,A_t) \ ← \ Q(S_t,A_t)+α[R_{t+1}+\gamma \underset{a}{\max} Q(S_{t+1},a)-Q(S_t,A_t)]$ &emsp;&emsp;&emsp;$(2)$ </center>

La Ecuación (1) es la regla de actualización que usa SARSA y la Ecuación (2) es la regla de actualización que utiliza Q-Learning. Note que la única diferencia entre ambas expresiones es un término de maximización sobre acciones que aparece en la Ecuación (2). Esto deriva de que SARSA es un algoritmo <i>on-policy</i> mientras que Q-Learning es <i>off-policy</i>. Esto quiere decir que SARSA genera un comportamiento en el ambiente con la misma política que mejora, lo cual tiene como limitación que debe mantener exploración (<i>soft</i>). Al contrario, en Q-Learning la política que se mejora es distinta a la política que genera el comportamiento, por lo cual se puede hacer (<i>greedy</i>). Los macroalgoritmos correspondientes se muestran en la Figura 2 y en la Figura 3:


![SARSA_tabular](https://raw.githubusercontent.com/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/main/Images/SARSA_tabular.png)

<center>Figura 2. Algoritmo de SARSA para control de política (versión tabular). [1]</center>

![QLearning_tabular](https://raw.githubusercontent.com/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/main/Images/QLearning_tabular.png)


<center>Figura 3. Algoritmo de Q-Learning para control de política (versión tabular). [1]</center>

Note que el problema de la versión tabular es que se necesita estimar $Q(S_t,A_t)$ para cada pareja estado-acción. Cuando el problema es muy grande, es decir, que tiene muchos estados y acciones posibles, hay muchas entradas en la tabla que deben actualizarse. Esto no solo ocupa un gran espacio en memoria, sino que también complica la actualización de muchas parejas estado-acción si la exploración no es apropiada y puede demandar una cantidad demasiado grande de cálculos, recursos computacionales y tiempo.  

Por otro lado, en la versión de aproximación de funciones de estos algoritmos, primero hay que definir qué tipo de aproximación se utilizará. En [1] aparecen muchas opciones para aproximar las funciones de valor, entre las cuales se encuentran una versión de parámetros lineales, polinomios, codificación, entre otros. No obstante, la alternativa más utilizada actualmente y probablemente la más útil y robusta, es el uso de redes neuronales profundas. Las redes neuronales permiten estimar funciones modificando la salida a partir de una entrada utilizando pesos asociados a cada neurona y funciones no lineales que, combinadas, pueden reproducir casi cualquier tipo de función si se utilizan suficientes neuronas (y de ahí el término profundas).  

Fue en el año 2013 que Mnih et al. [2] publicaron un paper en el cual se detalla la implementación de redes neuronales profundas para la ejecución del algoritmo de Q-Learning en el aprendizaje de juegos de Atari. En su momento, este paper impuso un nuevo estado del arte, ya que fue la primera vez que se logró trasladar la ventaja de usar redes neuronales que aprendieran de datos sensoriales crudos, como los pixeles de una imágen, al campo de aprendizaje por refuerzo con éxito. En la Figura 3 se muestra el algoritmo propuesto de Deep Q-Networks o DQN:

![DQN](https://raw.githubusercontent.com/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/main/Images/DQN.png)


<center>Figura 4. Algoritmo de DQN para control de política. [2]</center>

Por otro lado, Deep SARSA es un algoritmo que, aunque utiliza redes neuronales profundas para realizar un estimativo de la función de valor, no tiene un almacenamiento en memoria de las experiencias observadas, que es una de las ventajas de DQN señaladas en [2], y también se caracteriza por ser <i>on-policy</i>.


## Instalación de Librerías  

Corra el siguiente bloque de código para instalar las librerías requeridas en el tutorial.


In [3]:
#Descarga librerías no incluidas en Colab usando pip
!pip install tensorflow==2.12 keras==2.12 keras-rl2 #instala versión de tensorflow compatible y keras-rl2
!pip install gymnasium #gym
!pip install renderlab #para renderizar gym
!pip install ale-py #ale-py

#Importa estas librerías
import gym #importa gymnasyum
import numpy as np #importa numpy
import tensorflow as tf #importa tensorflow

from keras.models import Sequential #de keras importa el modelo secuencial de redes
from keras.layers import Dense, Activation, Flatten, Input #de keras importa tipos de capas de la red
from tensorflow.keras.optimizers import Adam #de keras importa Adam para solucionar el gradiente
from rl.agents.dqn import DQNAgent #de keras rl 2 importa el agente de de DQN
from rl.agents.sarsa import SARSAAgent #de keras rl 2 importa el agente de Deep SARSA
from rl.policy import BoltzmannQPolicy #de keras rl2 importa la política de Boltzmann Q
from rl.memory import SequentialMemory #de keras rl2 importa memoria secuencial

import gymnasium as gym #importa la libreria de gym con las simulaciones
import renderlab #importa renderlab para los videos

#Importa otras librerías básicas
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pandas as pd

#Limpia los registros generados
from IPython.display import clear_output
clear_output()

## Familiarización con el Entorno de Gym  
Contenido de la sección 4.  


## Métodos Tabulares  
Contenido de la sección 5.  


## Métodos con Redes Neuronales  
Contenido de la sección 6.  


## Comparación  
Contenido de la sección 7.  


## Conclusiones  
Contenido de la sección 8.

## Referencias

[1] Sutton, R. S. and Barto, A. G. (2018). Reinforcement Learning: An Introduction. The MIT Press, second edition.

[2] Mnih, V., Kavukcuoglu, K., Silver, D., Graves, A., Antonoglou, I., Wierstra, D., and Riedmiller, M. (2013). Playing atari with deep reinforcement learning. cite arxiv:1312.5602Comment: NIPS Deep Learning Workshop 2013.
